In [ ]:
###################
# set up the environment, nis, and image saving path
#####################
import os
from nis_util import *
from simple_detection import *

path_to_nis = 'C:\\Program Files\\NIS-Elements\\nis_ar.exe'
save_base_path = 'C:\\Users\\Nikon\\Documents\\David\\tmpOverview'

%matplotlib inline

In [ ]:
point = np.array((200, 200))
np.array(get_rotation_matrix(path_to_nis)).reshape((2,2)).dot(point)

mat = np.array(get_rotation_matrix(path_to_nis)).reshape((2,2))
# get resolution
(xres, yres, siz, mag) = get_resolution(path_to_nis)
# get fov
fov_x = xres * siz / mag
fov_y = yres * siz / mag

direction = [1, -1]

start = [0, 0]
psz = [1,1]

import logging

logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.DEBUG)

bbox_pix2unit2((0,0,1,1), start, psz, direction, [fov_y, fov_x], mat)

In [6]:
import logging
import numpy as np
from numpy.linalg import inv


def bbox_pix2unit2(bbox, start, pixsize, direction, fov, mat):
    logger = logging.getLogger(__name__)
    
    extra_offset = inv(mat).dot(np.array(list(reversed(fov)), dtype=float)/2) * np.array(list(reversed(direction)), dtype=float)
    start_ = np.array(start, dtype=float) + np.array(list(reversed(list(extra_offset))), dtype=float)
    
    logger.debug('extra offset: {}, new start: {}'.format(extra_offset, start_))
    
    (ymin, xmin, ymax, xmax) = bbox
    
    bbox_tr = np.apply_along_axis(lambda x: mat.dot(x), 1, np.array([[xmin, ymin], 
                                                           [xmin, ymax],
                                                           [xmax, ymin],
                                                           [xmax, ymax]], dtype=float))
    # TODO: WHY?
    bbox_tr *= -1
    
    min_ = np.apply_along_axis(np.min, 0, bbox_tr)
    max_ = np.apply_along_axis(np.max, 0, bbox_tr)
    
    logger.debug('new min: {}, new max: {}'.format(min_, max_))
    
    bbox_tr_arr = np.array([list(reversed(list(min_))), list(reversed(list(max_)))], dtype=float)
    
    res = (bbox_tr_arr * np.array(pixsize, dtype=float) *
            np.array(direction, dtype=float) + start_)
    
    logger.debug('bbox: {}, toUnit: {}'.format(bbox, res.reshape((4,))))
    return res.reshape((4,))


def do_scan(field, oc_overview, oc_detail, path_to_nis, save_base_path, prefix, suffix='.nd2', do_plot=True,
           z_range = 10, z_step=2, z_drive='Ti2 ZDrive', dry_run_details=False, stitched=True):
    
    logger = logging.getLogger(__name__)
    
    # get field and directions 
    (left, right, top, bottom) = field
    direction = [1 if top<bottom else -1, 1 if left<right else -1]
    
    # do overview
    set_optical_configuration(path_to_nis, oc_overview)
    
    # get resolution
    (xres, yres, siz, mag) = get_resolution(path_to_nis)
    # get fov
    fov_x = xres * siz / mag
    fov_y = yres * siz / mag
    
    logger.debug('overview resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
    
    ov_path = os.path.join(save_base_path, prefix + '_overview' + suffix)    
    do_large_image_scan(path_to_nis, ov_path, left, right, top, bottom)
    
    print('finished overview, detecting wings...')
    
    # detect wings
    img = read_bf(ov_path)
    # bboxes = detect_wings_simple(img, [top - fov_y * direction[0] / 2.0, left - fov_x * direction [1] / 2.0], [siz/mag,siz/mag], direction, plot=do_plot, layers=2)
    
    bboxes = detect_wings_simple(img, plot=do_plot, layers=2)
    
    mat = np.array(get_rotation_matrix(path_to_nis)).reshape((2,2))
    # pixels to units
    bboxes = [bbox_pix2unit2(b, [top, left], [siz/mag,siz/mag], direction,[fov_y, fov_x], mat) for b in bboxes]
    # bboxes = [scale_bbox(bbox, expand_factor=.15) for bbox in bboxes]
    
    print('detected {} wings:'.format(len(bboxes)))
    idx = 0
    for bbox in bboxes:
        print('scanning wing {}: {}'.format(idx, bbox))
        
        (ymin, xmin, ymax, xmax) = bbox
        wing_path = os.path.join(save_base_path, prefix + '_wing' + str(idx) + suffix) 
        idx += 1
        
        set_optical_configuration(path_to_nis, oc_detail)
        pos = get_position(path_to_nis)
        
        (xres, yres, siz, mag) = get_resolution(path_to_nis)
        fov = get_fov_from_res(get_resolution(path_to_nis))
        logger.debug('detail resolution: {}, {}, {}, {}'.format(xres, yres, siz, mag))
        logger.debug('fov: {}'.format(fov))
        
        grid = gen_grid(fov, [xmin, ymin], [xmax, ymax], 0.15, True)
        
        # get resolution
        
        # get fov
        fov_x = xres * siz / mag
        fov_y = yres * siz / mag
        # correct for half view offset (we add this time) - may not be neccessary at this magnification/fov
        # we should probably do that, ignore it for now
        # grid = [[g[0] + fov_x * direction[1] / 2.0, g[1] + fov_y * direction [0] / 2.0] for g in grid]
        if not stitched:
            for g in grid:
                logger.debug('wing {}: will scan tile at {}'.format(idx-1, g))
        
        if not dry_run_details:
            if not stitched:
                nda = NDAcquisition(wing_path)
                nda.set_z(int(z_range/2), int(z_range/2), int(z_step), z_drive)
                nda.add_points(map(lambda x : (x[0], x[1], pos[2] - pos[3]), grid))
                nda.prepare(path_to_nis)
                nda.run(path_to_nis)
            else:
                do_large_image_scan(path_to_nis, wing_path, xmax, xmin, ymin, ymax, 15, True)

In [7]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.DEBUG)

phi = np.arccos(0.995) - 2 * np.pi
bbox_pix2unit2((0,0,100,100), [0.0,0.0], [1.0,1.0], [1, -1], [100.0, 100.0], np.array([[np.cos(phi), - np.sin(phi)],[np.sin(phi), np.cos(phi)]], dtype=float))

2017-12-18 10:35:58,030 - DEBUG: extra offset: [-54.74374609  44.75625391], new start: [ 44.75625391 -54.74374609]
2017-12-18 10:35:58,033 - DEBUG: new min: [ -99.5        -109.48749218], new max: [ 9.98749218 -0.        ]
2017-12-18 10:35:58,034 - DEBUG: bbox: (0, 0, 100, 100), toUnit: [-64.73123827  44.75625391  44.75625391 -64.73123827]


array([-64.73123827,  44.75625391,  44.75625391, -64.73123827])

In [ ]:
get_fov_from_res(get_resolution(path_to_nis))
# get resolution
(xres, yres, siz, mag) = get_resolution(path_to_nis)
# get fov
fov_x = xres * siz / mag
fov_y = yres * siz / mag

(left, right, top, bottom) = (53331, 28806, -20726, 20464)
direction = [1 if top<bottom else -1, 1 if left<right else -1]
direction

In [ ]:
import logging

logging.basicConfig(format='%(asctime)s - %(levelname)s: %(message)s', level=logging.DEBUG)

slide_left = 'NG_Overview_009'
slide_mid = None #'NG_Overview_024'
slide_right = None#'NG_Overview_025'

do_scan_left  = slide_left != None
if do_scan_left:
    print('Scanning left scan.')
    do_scan((53331, 28806, -20726, 20464), 'DIA4x', 'DIA10x', path_to_nis, save_base_path, slide_left, dry_run_details=True )
else:
    print('Skipping left slide.')

# mid slide
do_scan_mid  = slide_mid != None

if do_scan_mid:
    print('Scanning mid scan.')
    do_scan((13400, -7850, -20954, 18220), 'DIA4x', 'DIA10x', path_to_nis, save_base_path, slide_left )
else:
    print('Skipping middle slide.')

# right slide
do_scan_right  = slide_right != None
if do_scan_right:
    print('Scanning right scan.')
    do_scan((-26500, -50000, -21053, 18177), 'DIA4x', 'DIA10x', path_to_nis, save_base_path, slide_left )
else:
    print('Skipping right slide.')